<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Regression_LargeDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As the classification models have had some difficulties classifying the fires accurately, we will attempt to build regression models (on Acres Burned feature) instead and see if these have more success.

I will deploy the following techniques:

1.   One-Hot Encoding of Categorical Variables
2.   Splitting the Training and Testing data
3.   Feature Selection
4.   Normalize Data
5.   Regressions




In [3457]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Read in the smaller of the final datasets. The dataset contains around 1200 fires from 2016-2019
wildfire_set1 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfire_set1_w_allfeatures.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3458]:
wildfire_set1.head()

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens,Has_Elevation,Elevation
0,0,2005,FOUNTAIN,0.10,A,Miscellaneous,Plumas National Forest,1.0,Feb,33.0,...,0.0,19.0,1.0,eureka_lake,Feather,79.6,34.0,0.43,1,2965.8432
1,1,2004,PIGEON,0.25,A,Lightning,Eldorado National Forest,1.0,May,133.0,...,0.0,3.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,6207.2736
2,2,2004,SLACK,0.10,A,Debris Burning,Eldorado National Forest,1.0,Jun,152.0,...,0.0,11.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,3454.6824
3,3,2004,DEER,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7759.0920
4,4,2004,STEVENOT,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7598.3328


In [3459]:
print(wildfire_set1.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens', 'Has_Elevation', 'Elevation'],
      dtype='object')


Drop all columns that will not be relevant for our regression task:Unnamed: 0, Year, Name, Cause, SOURCE_REPORTING UNIT NAME, DaysBurn, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [3460]:
wildfire_set1.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'Cause', 'Fire Size Rank',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn','Contained Month', 'Discovered DOY',
                      'Contained DOY','Latitude','Longitude','County','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin','Has_Elevation'], inplace = True)

Taking a look at the dataset now that we have dropped the nonrelevant columns

In [3461]:
wildfire_set1.head(5)

,AcresBurned,Discovery Month,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens,Elevation
0,0.10,Feb,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,49.06,0.0,19.0,1.0,79.6,34.0,0.43,2965.8432
1,0.25,May,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,14.76,0.0,3.0,1.0,108.6,38.1,0.35,6207.2736
2,0.10,Jun,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,40.37,0.0,11.0,1.0,108.6,38.1,0.35,3454.6824
3,0.10,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7759.0920
4,0.10,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7598.3328


In [3462]:
wildfire_set1.shape

(114558, 19)

# Investigate the presence of null values in the dataset:
Below, we have the count of totall null values in each column for  wildfire_set1.

In [3463]:
print(wildfire_set1.isnull().sum())

AcresBurned           0
Discovery Month       0
AWND               6254
CLDD               3335
DP10               3042
DX90               3333
PRCP               3042
TAVG               3335
TMAX               3333
TMIN               3285
PRCP_6M            2470
PRCP_RS            2811
DX90_2M            2215
DP10_2M            1599
Receives Snow       102
Mar_SP              102
Mar_WC              102
Mar_Dens            102
Elevation             0
dtype: int64


In [3464]:
wildfire_set1 = wildfire_set1.dropna()

In the cell above, we drop all null values in the dataset. We are able to drop all nulls in this case because our dataset is sizeable with relatively few null values in each so not much data is lost in this process. Looking at the shape of the dataframe after the drops, we see that the dataset remains a similar size post drop.

In [3465]:
#we lose around 50 values when dropping NaN's
wildfire_set1.shape

(106024, 19)

# For our regressions, we will try to predict the number of Acres Burned in the fire

In [3466]:
print(wildfire_set1.columns)

Index(['AcresBurned', 'Discovery Month', 'AWND', 'CLDD', 'DP10', 'DX90',
       'PRCP', 'TAVG', 'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M',
       'DP10_2M', 'Receives Snow', 'Mar_SP', 'Mar_WC', 'Mar_Dens',
       'Elevation'],
      dtype='object')


Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides discovery and fire size rank are numerical (of type float or int) so all we have to do is one hot encode the discovery month of the fire.

In [3467]:
wildfire_set1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106024 entries, 0 to 114557
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AcresBurned      106024 non-null  float64
 1   Discovery Month  106024 non-null  object 
 2   AWND             106024 non-null  float64
 3   CLDD             106024 non-null  float64
 4   DP10             106024 non-null  float64
 5   DX90             106024 non-null  float64
 6   PRCP             106024 non-null  float64
 7   TAVG             106024 non-null  float64
 8   TMAX             106024 non-null  float64
 9   TMIN             106024 non-null  float64
 10  PRCP_6M          106024 non-null  float64
 11  PRCP_RS          106024 non-null  float64
 12  DX90_2M          106024 non-null  float64
 13  DP10_2M          106024 non-null  float64
 14  Receives Snow    106024 non-null  float64
 15  Mar_SP           106024 non-null  float64
 16  Mar_WC           106024 non-null  floa

# Below we use the describe function to get a general outlook on mean, max, min, and percentiles for each of the numerical features in the dataset

In [3468]:
wildfire_set1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
AcresBurned,106024.0,79.390867,2456.308820,0.0010,0.1000,0.2500,1.0000,315578.8000
AWND,106024.0,6.294384,1.875589,0.4000,4.9000,6.0000,7.4000,17.2000
CLDD,106024.0,224.058279,215.023594,0.0000,19.0000,180.0000,377.0000,1113.0000
DP10,106024.0,0.940853,1.810402,0.0000,0.0000,0.0000,1.0000,24.0000
DX90,106024.0,12.191391,11.505412,0.0000,0.0000,9.0000,23.0000,31.0000
PRCP,106024.0,0.457943,1.135417,0.0000,0.0000,0.0400,0.4100,33.7000
TAVG,106024.0,69.135184,10.958172,18.3000,61.4000,70.4000,77.4000,100.9000
TMAX,106024.0,83.236084,13.094602,33.0000,73.8000,85.1000,93.8000,114.3000
TMIN,106024.0,55.033364,9.797339,3.7000,49.0000,55.8000,61.8000,89.0000
PRCP_6M,106024.0,9.410894,9.301400,0.0000,2.8700,6.5900,12.9100,85.9100


# Some Interesting Takeaways:
*   Acres Burned has an average of 5200 acres yet the 50th percentile fire only has 87 acres burned. This tells us there are some extremely massive fires in this dataset contributing to this large mean acres burned
*   DP10 and PRCP look to not be the best distinguishing features as 0-50th percentile for both all have 0. This can be attributed to the vast majority of the state of California having undergone extreme drought conditions during the last decade (with some exceptions in rainy coastal areas in Northern California)
*  The temperature features seem to have a wide spread which is good
*  Three of our engineered weather features DX90_2M, PRCP_6M, PRCP_RS seem to have a healthy spread and diverse values which is good
*   Average Elevation that fires in the dataset occurred at is 1760 ft. with 75% occuring under 2398 ft. Interesting to see that most fires begin at lower elevations







# One-Hot Encoding Categorical Variables


*   Encode the discovery month (the only categorical variable) in both datasets



In [3469]:
#Add the dummy columns to the dataset
dummy_month = pd.get_dummies(wildfire_set1['Discovery Month'])
wildfire_set1 = pd.merge(left = wildfire_set1, right = dummy_month, left_index = True, right_index = True)
wildfire_set1 = wildfire_set1.drop(columns = ['Discovery Month'])
wildfire_set1.head(10)

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,0.10,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,...,0,1,0,0,0,0,0,0,0,0
1,0.25,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,...,0,0,0,0,0,0,1,0,0,0
2,0.10,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,...,0,0,0,0,1,0,0,0,0,0
3,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
4,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
5,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
6,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
7,0.80,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
8,1.00,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
9,0.10,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0


In [3470]:
wildfire_set1.head()

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,0.10,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,...,0,1,0,0,0,0,0,0,0,0
1,0.25,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,...,0,0,0,0,0,0,1,0,0,0
2,0.10,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,...,0,0,0,0,1,0,0,0,0,0
3,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
4,0.10,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0


In [3471]:
#Write the clean, encoded, and randomly shuffled data to csv before continuing 
wildfire_set1.to_csv('wildfire1_regression_clean.csv')

Split the data into train-test sets

In [3472]:
#We will train our models using the more recent dataset

In [3473]:
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [3474]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay

Utilize an 80-20 train test split on our large dataset

In [3475]:
train_data, test_data = train_test_split(wildfire_set1, test_size = 0.2, random_state = 1)

Print the shape of training and testing datasets after the split to make sure we have done this correctly

In [3476]:
print(train_data.shape)
print(test_data.shape)

(84819, 30)
(21205, 30)


# Feature Selection

In [3477]:
from sklearn import linear_model

In [3478]:
from sklearn.feature_selection import SelectKBest, f_regression

In [3479]:
top_features_ordered = []

In [3480]:
train_target = train_data['AcresBurned']
train_features = train_data.drop(columns = ['AcresBurned'])

In [3481]:
test_target = test_data['AcresBurned']
test_features = test_data.drop(columns = ['AcresBurned'])

Select the 6 best features using sklearn's SelectKBest (leads to lowest MAE)

In [3482]:
best_features = SelectKBest(f_regression, k=10)
k_best_data = best_features.fit_transform(train_features, train_target)
mask = best_features.get_support()
new_features = train_features.columns[mask]
for item in new_features:
  if item not in top_features_ordered:
    top_features_ordered.append(item)

In [3483]:
count = 1
for item in top_features_ordered:
  print(count, '. ', item)
  count += 1

1 .  DP10
2 .  PRCP
3 .  TAVG
4 .  TMIN
5 .  PRCP_6M
6 .  DX90_2M
7 .  DP10_2M
8 .  Elevation
9 .  May
10 .  Oct


# Now we normalize the train and test sets independently

In [3484]:
#train_data_selected_X = train_features[top_features_ordered] k = 6, lasso MAE= 148
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Aug','May','Oct']] linear mae = 131
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct','Apr']] #linear mae = 132 
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct']] linear mae = 130
#train_data_selected_X = train_features[['PRCP_6M', 'TMIN','DX90_2M','Elevation','Oct','May']] linear mae = 133
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Jul','May','Oct']] linear mae = 130
#train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Apr','May','Oct']] #linear mae = 125

train_data_selected_X = train_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct']] #linear mae = 123

train_data_y = train_target

#test_data_selected_X = test_features[top_features_ordered]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Aug','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct','Apr']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TMIN','DX90_2M','Elevation','Oct','May']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Elevation','Jul','May','Oct']]
#test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Apr','May','Oct']]

test_data_selected_X = test_features[['PRCP_6M','TAVG','TMIN','DX90_2M','Jul','May','Oct']]

test_data_y = test_target


#crucial features - PRCP_6M, May, Oct, TMIN, TAVG, DX90_2M, elevation
#neutral - PRCP
#less important
#not important - Aug, Apr

In [3485]:
print('Train Features:')
print('train_data_X dimensions: ', train_data_selected_X.shape)
print('train_data_y dimensions:', train_data_y.shape, '\n')

print('Test Features:')
print('test_data_X dimensions: ', test_data_selected_X.shape)
print('test_data_y dimensions:', test_data_y.shape, '\n')

Train Features:
train_data_X dimensions:  (84819, 7)
train_data_y dimensions: (84819,) 

Test Features:
test_data_X dimensions:  (21205, 7)
test_data_y dimensions: (21205,) 



Normalize data using min-max-scalar (every feature will normalized so that range of values is 0-1) after splitting into train and test

In [3486]:
from sklearn.preprocessing import MinMaxScaler

In [3487]:
#utilize minmax scaler normalization
scaler = MinMaxScaler()

#Normalize the train, validate, and test independently after splitting
train_data_X = scaler.fit_transform(train_data_selected_X)
#train_target = scaler(train_target)

test_data_X = scaler.fit_transform(test_data_selected_X)
#train_target = scaler(test_target)

# Fit the model using our normalized features to predict Acres Burned

In [3488]:
multivariate_regression = linear_model.LinearRegression()
multivariate_regression.fit(train_data_X, train_data_y)

LinearRegression()

Model coefficients: (Beta's)

In [3489]:
multivariate_regression.coef_

array([-142.76436105, -184.2918476 ,  436.43100465,  -23.76192593,
        -59.97521109,  -59.86761478,  114.84798504])

Model intercept:

In [3490]:
multivariate_regression.intercept_

-41.939447459870735

In [3491]:
from sklearn import metrics
import numpy as np

In [3492]:
y_predicted = multivariate_regression.predict(test_data_X)

In [3493]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_predicted), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_predicted), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_predicted)))

Mean Squared Error:  1741465.218079261 

Mean Absolute Error:  125.22872195740082 

Root Mean Squared Error:  1319.6458684356426


Note: The multivariate linear regression performs MUCH better on the large dataset with a MAE of 142 acres vs. that of 137 acres when trained on the small dataset

Try Ridge and Lasso Regressions to see if these outperform standard multivariate linear regression:

# Ridge Regression:

In [3494]:
from sklearn.linear_model import Ridge

In [3495]:
ridge_regression = Ridge(alpha = 150)
ridge_regression.fit(train_data_X, train_data_y)

Ridge(alpha=150)

In [3496]:
ridge_regression.coef_

array([-100.46711781,   22.83815847,  189.09323539,  -25.57699812,
        -60.54139627,  -61.06498114,  115.18033947])

In [3497]:
ridge_regression.intercept_

-24.530233565630937

In [3498]:
y_predicted = ridge_regression.predict(test_data_X)

# MAE for Ridge Regression is slightly lower

In [3499]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_predicted), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_predicted), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_predicted)))

Mean Squared Error:  1741600.2773384107 

Mean Absolute Error:  128.26672527213765 

Root Mean Squared Error:  1319.6970399824388


# Lasso Regression:

In [3500]:
from sklearn.linear_model import Lasso

In [3501]:
lasso_regression = Lasso(alpha = 1.5)
lasso_regression.fit(train_data_X, train_data_y)

Lasso(alpha=1.5)

In [3502]:
lasso_regression.coef_

array([ -0.        ,   0.        ,  90.97853377,   4.72846805,
       -37.25164534, -46.29062065,  91.67472423])

In [3503]:
lasso_regression.intercept_

27.78890353302775

In [3504]:
y_predicted = lasso_regression.predict(test_data_X)

In [3505]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_predicted), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_predicted), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_predicted)))

Mean Squared Error:  1741057.764006504 

Mean Absolute Error:  131.39070452667147 

Root Mean Squared Error:  1319.4914793231915


# Decision Tree Regressor

In [3506]:
from sklearn.tree import DecisionTreeRegressor

In [3507]:
decision_tree = DecisionTreeRegressor(max_depth= 3)

In [3508]:
decision_tree.fit(train_data_selected_X, train_data_y)

DecisionTreeRegressor(max_depth=3)

In [3509]:
y_pred_dt = decision_tree.predict(test_data_selected_X)

In [3510]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_pred_dt), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_pred_dt), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_pred_dt)))

Mean Squared Error:  1788993.269111853 

Mean Absolute Error:  135.5919792405282 

Root Mean Squared Error:  1337.5325301135122


#Deep Neural Network Regressor (using 6 best features):

In [779]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

In [780]:
print('Train Features:')
print('train_data_X dimensions: ', train_data_selected_X.shape)
print('train_data_y dimensions:', train_data_y.shape, '\n')

print('Test Features:')
print('test_data_X dimensions: ', test_data_selected_X.shape)
print('test_data_y dimensions:', test_data_y.shape, '\n')

Train Features:
train_data_X dimensions:  (84819, 8)
train_data_y dimensions: (84819,) 

Test Features:
test_data_X dimensions:  (21205, 8)
test_data_y dimensions: (21205,) 



# Neural Network Regressor #1:

In [781]:
tf.random.set_seed(5)

network1 = tf.keras.models.Sequential()
network1.add(tf.keras.Input(shape=(6,)))
network1.add(tf.keras.layers.Dense(15, activation='relu'))
network1.add(tf.keras.layers.Dense(15, activation='tanh'))
network1.add(tf.keras.layers.Dense(15, activation='relu'))
network1.add(tf.keras.layers.Dense(1))

network1.compile(
   loss=tf.keras.losses.MeanAbsoluteError(),
   optimizer=tf.keras.optimizers.Adam(),
   metrics='mean_absolute_error')

history1 = network1.fit(train_data_X, train_data_y, epochs=10,validation_data=(test_data_X,test_data_y))

Epoch 1/10


ValueError: ignored

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history1.history['mean_absolute_error'])
plt.plot(history1.history['val_mean_absolute_error'])
plt.title('Model MAE')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross-Validation'], loc='upper right')
plt.show()

# Neural Network Regressor #2:

In [ ]:
tf.random.set_seed(5)

network = tf.keras.models.Sequential()
network.add(tf.keras.Input(shape=(6,)))
network.add(tf.keras.layers.Dense(8, activation='relu'))
network.add(tf.keras.layers.Dense(8, activation='tanh'))
network.add(tf.keras.layers.Dense(8, activation='relu'))
network.add(tf.keras.layers.Dense(1))

network.compile(
   loss=tf.keras.losses.MeanAbsoluteError(),
   optimizer=tf.keras.optimizers.Adam(),
   metrics='mean_absolute_error')

history = network.fit(train_data_X, train_data_y, epochs=10,validation_data=(test_data_X,test_data_y))

I'm a little confused as this model does not learn at all. However, it performs very well

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Model MAE')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross-Validation'], loc='upper right')
plt.show()

I experimented with different layers, activation functions, and but all gave pretty much identical results. They all perform well but models are not learning at all.

In [ ]:
tf.random.set_seed(5)

network2 = tf.keras.models.Sequential()
network2.add(tf.keras.Input(shape=(6,)))
network2.add(tf.keras.layers.Dense(20, activation='relu'))
network2.add(tf.keras.layers.Dense(1))

network2.compile(
   loss=tf.keras.losses.MeanAbsoluteError(),
   optimizer=tf.keras.optimizers.Adam(),
   metrics='mean_absolute_error')

history2 = network2.fit(train_data_X, train_data_y, epochs=10,validation_data=(test_data_X,test_data_y))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history2.history['mean_absolute_error'])
plt.plot(history2.history['val_mean_absolute_error'])
plt.title('Model MAE')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross-Validation'], loc='upper right')
plt.show()